In [1]:
import cartopy.crs as ccrs
import hvplot.xarray
import xarray as xr
import requests
from glob import glob
import pandas as pd
from herbie import Herbie
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map

In [2]:
for year in range(2000, 2020):
    dates = xr.open_dataset(f"NZ_GEFS_{year}.nc").time.to_pandas()
    #print(year, len(dates))
    expected = pd.date_range(f"{year}-01-01 03:00", f"{year+1}-01-01", freq="3h")
    missing = expected[~expected.isin(dates)]
    extra = dates[~dates.isin(expected)]
    if len(missing) > 0:
        print("Missing dates:", len(missing), missing)

Missing dates: 56 DatetimeIndex(['2000-01-01 03:00:00', '2000-01-01 06:00:00',
               '2000-01-01 09:00:00', '2000-01-01 12:00:00',
               '2000-01-01 15:00:00', '2000-01-01 18:00:00',
               '2000-01-01 21:00:00', '2000-01-02 00:00:00',
               '2000-01-02 03:00:00', '2000-01-02 06:00:00',
               '2000-01-02 09:00:00', '2000-01-02 12:00:00',
               '2000-01-02 15:00:00', '2000-01-02 18:00:00',
               '2000-01-02 21:00:00', '2000-01-03 00:00:00',
               '2000-01-03 03:00:00', '2000-01-03 06:00:00',
               '2000-01-03 09:00:00', '2000-01-03 12:00:00',
               '2000-01-03 15:00:00', '2000-01-03 18:00:00',
               '2000-01-03 21:00:00', '2000-01-04 00:00:00',
               '2000-01-04 03:00:00', '2000-01-04 06:00:00',
               '2000-01-04 09:00:00', '2000-01-04 12:00:00',
               '2000-01-04 15:00:00', '2000-01-04 18:00:00',
               '2000-01-04 21:00:00', '2000-01-05 00:00:00',
      

In [8]:
def download_GEFS(date):
    try:
        H = Herbie(date, model="gefs_wave_reforecast", member=1, save_dir="./GEFS", verbose=False)
        ds = H.xarray(rf":(?:HTSGW|DIRPW|PERPW):surface:(?:anl|(?:3|6|9|12|15|18|21) hour fcst)", remove_grib=True, backend_kwargs=dict(decode_timedelta=False))
        ds = ds.sel(longitude=slice(165, 180), latitude=slice(-33,-48))
        ds = ds.drop_vars("time").rename({"step": "time"}).assign_coords(time=ds.valid_time.values).drop_vars(["gribfile_projection", "surface", "valid_time"])
        return ds
    except:
        print(f"Failed to download data for {date}")
        return None

missing_dates = ["2009-10-19", "2010-01-08", "2010-01-09", "2011-04-10", "2014-03-13", "2018-06-16", "2019-04-14", "2019-04-30"]
dsets = thread_map(
    download_GEFS,
    missing_dates,
    max_workers=len(missing_dates),
    desc=f"Downloading GEFS data",
    unit="day",
)
dsets = [ds for ds in dsets if ds is not None]
dsets = xr.concat(dsets, dim="time")
dsets.to_netcdf(f"NZ_GEFS_missing.nc", mode="w")

In [12]:
dsets = xr.open_mfdataset("NZ_GEFS_*.nc", combine="nested")
dsets

<xarray.Dataset> Size: 3GB
Dimensions:    (latitude: 61, longitude: 60, time: 58384)
Coordinates:
  * latitude   (latitude) float64 488B -33.0 -33.25 -33.5 ... -47.5 -47.75 -48.0
  * longitude  (longitude) float64 480B 165.0 165.3 165.5 ... 179.3 179.5 179.8
  * time       (time) datetime64[ns] 467kB 2000-01-08T03:00:00 ... 2020-01-01
Data variables:
    swh        (time, latitude, longitude) float32 855MB dask.array<chunksize=(64, 61, 60), meta=np.ndarray>
    dirpw      (time, latitude, longitude) float32 855MB dask.array<chunksize=(64, 61, 60), meta=np.ndarray>
    perpw      (time, latitude, longitude) float32 855MB dask.array<chunksize=(64, 61, 60), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   gefs_wave_reforecast
    product:                 GEFSv12/wave_reforecast
    description:             NOAA Wave Ensemble Reforecast
    remote_grib:             https://noaa-nws-gefswaves-reforecast-pds.s3.ama...
    local_grib:              GEFS/gefs_wave_reforecast/20000108/subset_b4ef68...
    search:                  :(?:HTSGW|DIRPW|PERPW):surface:(?:anl|(?:3|6|9|1...

In [15]:
dsets.to_netcdf("NZ_GEFS.nc", mode="w")